# Agent

# LLM alapú Ágens készítése a MedQuad adatbázishoz.

A MedQuad (Medical Question Answering Dataset) egy angol nyelvű orvosi adatbázis, amelyet elsősorban mesterséges intelligencia és természetes nyelvi feldolgozás (NLP) kutatásokhoz fejlesztettek ki. Ez az adatbázis kérdés-válasz párokat tartalmaz, amelyek orvosi témákra fókuszálnak, és célja, hogy segítse az automatizált rendszereket abban, hogy pontos és megbízható válaszokat adjanak egészségügyi kérdésekre. Az adatbázis tartalma általában orvosi szakirodalomból, klinikai információkból és hiteles forrásokból származik, például a National Institutes of Health (NIH) vagy más elismert egészségügyi szervezetek publikációiból.

## Függőségek telepítése

In [ ]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface bitsandbytes

## Csomagok importálása

In [ ]:
import os
import kagglehub
import pandas as pd

from tqdm import tqdm

from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from matplotlib import pyplot as plt

In [ ]:
# Download latest version
path = kagglehub.dataset_download(handle="googleai/dataset-metadata-for-cord19")

print("Path to dataset files:", path)

In [ ]:
os.listdir(path)

In [ ]:
filename_with_path = path + "/" + os.listdir(path)[0]
filename_with_path

In [ ]:
df_meta_cord19 = pd.read_csv(filename_with_path)
df_meta_cord19.head()

In [ ]:
df_meta_cord19.info()

In [ ]:
df_meta_cord19_filtered = df_meta_cord19[df_meta_cord19['description'].notnull()]
df_meta_cord19_filtered.info()

## Create a vector database using the LlamaIndex function library.

## Create Agent

In [ ]:
# Létrehozzuk a nyelvi modellt (LLM), amit az ágens fog használni.
llm = HuggingFaceLLM(
    model_name="colesmcintosh/Llama-3.2-1B-Instruct-Mango",       # Nyelvi modell beállítása
    tokenizer_name="colesmcintosh/Llama-3.2-1B-Instruct-Mango",   # Nyelvi modell tokenizátorának beállítása
    context_window=2048,                                          # Maximum token limit
    max_new_tokens=256,                                           # Válasz maximális hossza
    device_map="cuda:0",                                          # GPU használata,
    generate_kwargs={"temperature": 0.95, "do_sample": True},     # Ezek a paraméterek befolyásolják a modell válaszainak véletlenszerűségét és kreativitását.
)

In [ ]:
Settings.llm = llm

In [ ]:
# Létrehozzuk a chat motort, ami az ágens párbeszédéért felelős.
chat_engine = index.as_chat_engine(
    # Ez a paraméter beállítja, hogy a chat motor a korábban létrehozott vektoradatbázist használja a válaszokhoz.
    chat_mode="context",
    # Ez a paraméter beállítja a chat motor memóriáját. A ChatMemoryBuffer emlékszik a korábbi beszélgetésekre.
    memory=ChatMemoryBuffer.from_defaults(token_limit=32000),
    # Ez a paraméter beállítja a rendszerüzenetet, ami az ágens viselkedését befolyásolja. Ebben az esetben az ágens egy orvosi chatbot, amely a MedQuad adathalmaz alapján válaszol.
    system_prompt=(
        "You are a medical chatbot, able to have normal interactions. You only answre based on MedQuad dataset."
    )
)

In [ ]:
# Ez egy végtelen ciklus, ami lehetővé teszi, hogy folyamatosan kérdéseket tegyünk fel az ágensnek.
while True:
  query = input("> ")                                     # Bekérjük a felhasználótól a kérdést.
  if query.lower() == "quit":                             # Ha a felhasználó azt írja, hogy "quit", akkor a ciklus megszakad.
      break

  # Streamelt válasz
  print("Agent: ", end="", flush=True)                    # Kiírjuk, hogy az ágens válaszolni fog.
  response = chat_engine.stream_chat(query)               # Ez a sor elküldi a kérdést a chat motornak, és megkapjuk a választ.
  for token in response.response_gen:                     # Ez a ciklus kiírja a választ részleteiben (tokenekként) streamelve.
      print(token, end="", flush=True)
  print()                                                 # Egy új sort írunk ki a válasz után.

chat_engine.reset()                                       # History törlése